In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
from mpmath import mp
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import math
import warnings
from ipywidgets import interact, interactive, fixed, interact_manual
plt.rcParams['figure.dpi'] = 180
plt.rcParams['figure.figsize'] = [12.0, 8.0]
plt.rcParams['text.latex.unicode'] = True
plt.rcParams['text.usetex'] = True
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

In [ ]:
log_sizes = np.array([i for i in range(1,35)])
sizes = np.array([2**i for i in range(1,35)])

In [ ]:
# Register file size is speculative
# Defaults are for AWS EC2 c5.2xlarge, an "Intel(R) Xeon(R) Platinum 8124M CPU @ 3.00GHz"
def init_plot(reg_file=128 * 8, l1=32*1024, l2=1024**2, l3=24 * 1024**2, ram=16 * 1024**3):
    plt.xlabel('Size $\\left[\\log_2 n\\right]$')
    plt.xscale('log')
    plt.xticks(sizes, [str(n) for n in log_sizes])
    plt.gca().xaxis.set_minor_locator(plt.NullLocator())
    
    def mem_line(size, label):
        plt.axvline(size / 32, color='grey', linestyle='--')
        plt.text(size / 32, 100, label)
    
    mem_line(reg_file, "REG")
    mem_line(l1, "L1")
    mem_line(l2, "L2")
    mem_line(l3, "L3")
    mem_line(ram, "RAM")

    plt.yscale('log')
    plt.ylabel('Time $\\left[\\mathtt{sec}\\right]$')

In [ ]:
def series(data, color='tab:blue'):
    plt.plot(sizes[:len(data)], data, color=color, marker='.')

## Benchmark c5.2xlarge

On AWS EC2 instance type `c5.2xlarge`. Root drive size increased to 256GB and a 64GB swapfile is added. 

In [ ]:
fft_heap = np.fromstring("""
1	0.000322729
2	0.000034127
3	0.000039599
4	0.000032414
5	0.000040861
6	0.00004389
7	0.000059015
8	0.000057413
9	0.000081098
10	0.000101278
11	0.000209461
12	0.000332409
13	0.000819755
14	0.00109282
15	0.003354192
16	0.00465073
17	0.015117839
18	0.020831895
19	0.071238697
20	0.092177723
21	0.297560154
22	0.404474188
23	1.258386787
24	1.751804361
25	5.333097693
26	7.724865138
27	22.811634182
28	33.926917188
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
fft_mmap = np.fromstring("""
1	0.000343832
2	0.000026941
3	0.000025195
4	0.000033528
5	0.000039073
6	0.000041008
7	0.00005011
8	0.000252068
9	0.000104768
10	0.00009109
11	0.000213909
12	0.000310817
13	0.000857938
14	0.001107311
15	0.003381245
16	0.004669626
17	0.015172413
18	0.020381842
19	0.07097182
20	0.091868017
21	0.300869807
22	0.406127257
23	1.263769443
24	1.758742483
25	5.507684328
26	9.187671884
27	56.873782122
28	121.002103585
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
transpose_heap = np.fromstring("""
1	0.000000321
2	0.000000132
3	0.000000339
4	0.000000112
5	0.00000033
6	0.000000165
7	0.000004809
8	0.000000552
9	0.000006833
10	0.00000152
11	0.000039416
12	0.000005317
13	0.000167323
14	0.000027488
15	0.000708976
16	0.000153153
17	0.003498693
18	0.001238266
19	0.01585475
20	0.005688364
21	0.063396217
22	0.025159004
23	0.255140885
24	0.104978946
25	1.034405426
26	0.432174938
27	4.143390446
28	1.809782571
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
transpose_mmap = np.fromstring("""
1	0.000000808
2	0.000000186
3	0.000000474
4	0.000000144
5	0.000000652
6	0.000000197
7	0.000001086
8	0.000000582
9	0.000009524
10	0.00000173
11	0.000023314
12	0.000006388
13	0.000146607
14	0.000030903
15	0.000726127
16	0.000164394
17	0.003318322
18	0.001002092
19	0.015842278
20	0.005797671
21	0.065393661
22	0.026388074
23	0.266523877
24	0.107756829
25	1.1331754649999999
26	0.86950222
27	18.279631663
28	32.567607439
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
permute_heap = np.fromstring("""
1	0.000000223
2	0.000000066
3	0.000000077
4	0.000000185
5	0.000000321
6	0.000000552
7	0.000000846
8	0.000001995
9	0.000003347
10	0.000004168
11	0.000008545
12	0.000018388
13	0.000033531
14	0.000073131
15	0.000170725
16	0.000416589
17	0.001399756
18	0.003641116
19	0.009271187
20	0.020368032
21	0.044254246
22	0.094244237
23	0.197541416
24	0.435237952
25	0.92625859
26	1.921617097
27	4.23983216
28	11.126569847
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
permute_mmap = np.fromstring("""
1	0.000000392
2	0.000000104
3	0.000000104
4	0.000000163
5	0.000000265
6	0.000000398
7	0.000000748
8	0.00000123
9	0.000002042
10	0.000003788
11	0.000007325
12	0.000020224
13	0.000040598
14	0.0000717
15	0.000161652
16	0.000402458
17	0.00102328
18	0.003096936
19	0.00939244
20	0.020978134
21	0.044792625
22	0.095683949
23	0.203506656
24	0.450967586
25	0.994789424
26	3.722098624
27	299.942189901
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
init_plot()
series(fft_heap, 'tab:red')
series(fft_mmap, 'tab:blue')
series(transpose_heap, 'tab:orange')
series(transpose_mmap, 'tab:cyan')
series(permute_heap, 'tab:green')
series(permute_mmap, 'tab:olive')

## Benchmark c5.2xlarge (64MiB RAM)

On AWS EC2 instance type `c5.2xlarge`. Root drive size increased to 256GiB and a 64GiB swapfile is added.

```
L1 Instruction-Cache: (32 KiB, 8-way associativity, direct-mapped)
L1 Data-Cache: (32 KiB, 8-way associativity, direct-mapped)
L2 Unified-Cache: (1024 KiB, 16-way associativity, direct-mapped)
L3 Unified-Cache: (24 MiB, 11-way associativity, hash-based-mapping)
```

Memory is restricted to 64MiB RAM using cgroups:

```
sudo cgcreate -t $USER:$USER -a $USER:$USER -g memory:limited
echo 67108864 > /sys/fs/cgroup/memory/limited/memory.limit_in_bytes
cgexec -g memory:limited ./fft
```

In [ ]:
fft_heap = np.fromstring("""
1	0.00034372
2	0.000039486
3	0.000035912
4	0.000037605
5	0.000041504
6	0.000042154
7	0.000055761
8	0.000055802
9	0.000086509
10	0.000104544
11	0.000249147
12	0.000277132
13	0.000917944
14	0.001096818
15	0.00341159
16	0.004738028
17	0.015229078
18	0.020695995
19	0.070773651
20	0.091959886
21	30.928260041
22	40.091185241
23	163.181767742
24	184.1793349
25	832.334289622
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
fft_mmap = np.fromstring("""
1	0.000327282
2	0.000035651
3	0.000028616
4	0.000040656
5	0.000043662
6	0.000036332
7	0.000054138
8	0.000056564
9	0.000078225
10	0.000108604
11	0.000244958
12	0.000287139
13	0.000859705
14	0.001116579
15	0.003552164
16	0.004715079
17	0.015413409
18	0.021136953
19	0.073038566
20	0.092504724
21	76.226779587
22	68.887253346
23	726.08383654
24	527.209818911
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
transpose_heap = np.fromstring("""
1	0.000000407
2	0.000000201
3	0.000000508
4	0.000000106
5	0.000000378
6	0.000000184
7	0.000004731
8	0.000000604
9	0.000007464
10	0.000001546
11	0.000031108
12	0.000005326
13	0.00012615
14	0.000028176
15	0.000708114
16	0.000138124
17	0.003178672
18	0.000945967
19	0.015287759
20	0.005595849
21	10.570897667
22	11.281882518
23	48.99322635
24	44.853152764
25	685.421398633
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
transpose_mmap = np.fromstring("""
1	0.000001035
2	0.000000202
3	0.000000511
4	0.000000152
5	0.000000656
6	0.000000288
7	0.000007064
8	0.000000804
9	0.000011185
10	0.000002899
11	0.000031892
12	0.00000538
13	0.000134255
14	0.000028246
15	0.000710412
16	0.000142058
17	0.003230439
18	0.001012398
19	0.015829714
20	0.005500206
21	10.533343567
22	11.358831156
23	48.803206975
24	44.780711319
25	186.202560067
26	145.289483772
""", sep=' ').reshape((-1, 2))[:,1]

In [ ]:
init_plot(ram=64*1024**2)
series(fft_heap, 'tab:red')
series(fft_mmap, 'tab:blue')
series(transpose_heap, 'tab:orange')
series(transpose_mmap, 'tab:cyan')